In [35]:
!rm ~/Downloads/*.csv

zsh:1: no matches found: /Users/leswing/Downloads/*.csv


In [18]:
# Save https://www.fantasypros.com/nfl/projections/qb.php?week=draft in folder "fantasy"
# Save https://www.fantasypros.com/nfl/adp/overall.php in folder "fantasy"
!mv /Users/leswing/Downloads/*.csv ./fantasy

In [68]:
import json
import pandas as pd

In [69]:
data = open('../app/public/projections.json').read()
d = json.loads(data)

In [70]:
bye_weeks = {
    "DET": 8,
    "LAC": 12,
    "PHI": 9,
    "TEN": 10,
    "KC": 10,
    "LAR": 8,
    "MIA": 12,
    "MIN": 6,
    "CHI": 5,
    "DAL": 10,
    "PIT": 5,
    "SF": 14,
    "CLE": 9,
    "GB": 5,
    "LV": 8,
    "SEA": 8,
    "ARI": 8,
    "CAR": 14,
    "NYG": 14,
    "TB": 9,
    "ATL": 5,
    "BUF": 7,
    "CIN": 10,
    "JAC": 8,
    "NO": 11,
    "NYJ": 9,
    "BAL": 7,
    "DEN": 12,
    "HOU": 6,
    "NE": 14,
    "WAS": 12,
    "IND": 11
}

def blank_player():
    # TODO fill out key, index, std, halfPpr, ppr
    return {
        'index': None,
        'key': None,
        'name': None,
        'pos': None,
        'team': None,
        'bye': 1,
        'std': None,
        'halfPpr': None,
        'ppr': None,
        'passYds': None,
        'rushTds': None,
        'dfSafeties': None,
        'kick4049': None,
        'dfSacks': None,
        'kick2029': None,
        'passInts': None,
        'rushYds': None,
        'receptionTds': None,
        'receptions': None,
        'dfInts': None,
        'kickExtraPoints': None,
        'passTds': None,
        'kick50': None,
        'fumbles': None,
        'kick019': None,
        'receptionYds': None,
        'twoPts': None,
        'dfPointsAllowedPerGame': None,
        'kick3039': None,
        'dfFumbles': None,
        'dfTds': None
    }
adp = pd.read_csv("fantasy/FantasyPros_2025_Overall_ADP_Rankings.csv")
adp = adp.dropna()
adp_lookup = {}
for _, row in adp.iterrows():
    key = f"{row['Player']}-{row['Team']}"
    if key not in adp_lookup:
        adp_lookup[key] = row['Rank']

In [71]:
def map_from_csv(df, column_mapping):
    qb_players = []
    for _, row in df.iterrows():
        new_player = blank_player()
        for k, v in row.items():
            if k not in column_mapping:
                continue
            new_key = column_mapping[k]
            try:
                my_v = str(v).replace(',','')
                new_player[new_key] = float(my_v)
            except:
                new_player[new_key] = v
        qb_players.append(new_player)
    return qb_players

def get_qbs():
    map_qb = {
        "Player": "name",
        "Team": "team",
        "YDS": "passYds",
        "TDS": "passTds",
        "INTS": "passInts",
        "YDS.1": "rushYds",
        "TDS.1": "rushTds",
        "FL": "fumbles",
        "POS": "pos"
    }
    df = pd.read_csv("fantasy/FantasyPros_2025_QB_ADP_Rankings.csv")
    df = df.dropna()
    df['POS'] = 'QB'
    return map_from_csv(df, map_qb)

def get_rbs():
    rb_map = {
        "Player": "name",
        "POS": 'pos',
        "Team": "team",
        "YDS": "rushYds",
        "TDS": 'rushTds',
        "REC": "receptions",
        "YDS.1": "receptionYds",
        "TDS.1": "receptionTds",
        "FL": "fumbles"
    }
    df = pd.read_csv("fantasy/FantasyPros_2025_RB_ADP_Rankings.csv")
    df = df.dropna(subset=['Overall'])
    df['POS'] = "RB"
    return map_from_csv(rb_df, rb_map)

def get_wrs():
    wr_map = {
        "Player": "name",
        "POS": 'pos',
        "Team": "team",
        "REC": "receptions",
        "YDS": "receptionYds",
        "TDS": "receptionTds",
        "YDS.1": "rushYds",
        "TDS.1": "rushTds",
        "FL": "fumbles"
    }
    df = pd.read_csv("fantasy/FantasyPros_2025_WR_ADP_Rankings.csv")
    df = df.dropna(subset=['Overall'])
    df['POS'] = 'WR'
    return map_from_csv(df, wr_map)

def get_tes():
    te_map = {
        "Player": "name",
        "POS": 'pos',
        "Team": "team",
        "REC": "receptions",
        "YDS": "receptionYds",
        "FL": "fumbles"
    }
    df = pd.read_csv("fantasy/FantasyPros_2025_TE_ADP_Rankings.csv")
    df = df.dropna(subset=['Overall'])
    df['POS'] = 'TE'
    return map_from_csv(df, te_map)

def get_ks():
    k_map = {
        "Player": "name",
        "POS": 'pos',
        "Team": "team",
        "XPT": "kickExtraPoints",
        "FG": "kick019"
    }
    df = pd.read_csv("fantasy/FantasyPros_2025_K_ADP_Rankings.csv")
    df = df.dropna(subset=['Overall'])
    df['POS'] = 'K'
    return map_from_csv(df, k_map)

def get_ds():
    dst_map = {
        "Player": "name",
        "POS": 'pos',
        "Team": "team",
        "SACK": "dfSacks",
        "INT": "dfInts",
        "FR": "dfFumbles",
        "TD": "dfTds",
    }
    df = pd.read_csv("fantasy/FantasyPros_2025_DST_ADP_Rankings.csv")
    df = df.drop("Team", axis=1)
    df = df.dropna(subset=['AVG'])
    df['POS'] = 'DST'
    return map_from_csv(df, dst_map)

def get_adp_for_player(p):
    key = f"{p['name']}-{p['team']}"
    if key not in adp_lookup:
        return 1_000
    return adp_lookup[key]

In [72]:
qb_players = get_qbs()
rb_players = get_rbs()
wr_players = get_wrs()
te_players = get_tes()
k_players = get_ks()
dst_players = get_ds()
[len(x) for x in [qb_players, rb_players, wr_players, te_players, k_players, dst_players]]

[28, 97, 114, 42, 22, 32]

In [73]:
all_players = []
all_players.extend(qb_players)
all_players.extend(rb_players)
all_players.extend(wr_players)
all_players.extend(te_players)
all_players.extend(k_players)
all_players.extend(dst_players)

In [74]:
import uuid
#key, index, std, halfPpr, ppr
for i, p in enumerate(all_players):
    p['key'] = str(uuid.uuid4())
    p['index'] = i + 1
    p['std'] = get_adp_for_player(p)
    p['halfPpr'] = get_adp_for_player(p)
    p['ppr'] = get_adp_for_player(p)
    try:
        p['bye'] = bye_weeks[p['team']]
    except:
        pass

In [75]:
all_players = sorted(all_players, key=lambda x: x['ppr'])
d['data'] = all_players
len(all_players)

335

In [76]:
with open("projections.json", 'w') as fout:
    fout.write(json.dumps(d))

In [77]:
all_players

[{'index': 1,
  'key': '5b121a76-3820-42a2-a1ab-39c5b07ba87f',
  'name': 'Lamar Jackson',
  'pos': 'QB',
  'team': 'BAL',
  'bye': 7,
  'std': 1000,
  'halfPpr': 1000,
  'ppr': 1000,
  'passYds': None,
  'rushTds': None,
  'dfSafeties': None,
  'kick4049': None,
  'dfSacks': None,
  'kick2029': None,
  'passInts': None,
  'rushYds': None,
  'receptionTds': None,
  'receptions': None,
  'dfInts': None,
  'kickExtraPoints': None,
  'passTds': None,
  'kick50': None,
  'fumbles': None,
  'kick019': None,
  'receptionYds': None,
  'twoPts': None,
  'dfPointsAllowedPerGame': None,
  'kick3039': None,
  'dfFumbles': None,
  'dfTds': None},
 {'index': 2,
  'key': '26b07818-19cf-452f-bb42-aecc25414b6d',
  'name': 'Josh Allen',
  'pos': 'QB',
  'team': 'BUF',
  'bye': 7,
  'std': 1000,
  'halfPpr': 1000,
  'ppr': 1000,
  'passYds': None,
  'rushTds': None,
  'dfSafeties': None,
  'kick4049': None,
  'dfSacks': None,
  'kick2029': None,
  'passInts': None,
  'rushYds': None,
  'receptionTds': No

In [78]:
!cp ./projections.json ../app/public/projections.json